**DECISION TREE**

Importing libraries

In [6]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
from dataset import TimeSeriesDataset
from dataset import NumpyToTensor

Importing data

In [7]:
train_data = pd.read_csv("data/train_delhi_aqi.csv")
test_data = pd.read_csv("data/test_delhi_aqi.csv")
val_data = pd.read_csv("data/val_delhi_aqi.csv")

In [11]:
train_data.head()

,PM2_5,PM10,NO2,SO2,CO_ug,Ozone,Temp,RH,WS,BP,WD,AQI (PM2.5)
31125,239.00,325.75,59.83,8.65,0.88,58.50,23.098722,78.08,1.30,988.10,346.37,288.600000
36812,35.77,50.43,28.75,4.15,0.63,23.37,23.098722,62.50,0.50,982.00,84.00,100.925001
56176,20.56,45.89,5.00,15.50,0.88,12.00,23.970000,76.75,0.68,1041.00,320.00,68.290597
66679,31.00,67.50,5.98,1.00,0.88,4.00,28.700000,97.97,0.34,991.52,228.25,90.598291
29992,80.63,350.94,70.74,0.90,0.88,17.35,18.590000,95.67,0.03,980.00,9.50,163.278946
